In [1]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d
from copy import deepcopy
from tqdm import tqdm

In [2]:
file = './data/136Xe_hsd_2ds.txt'
energy_threshold = 30

with open(file, 'r') as file:
    lines = file.readlines()
    extracted_data = [np.array(re.sub(' +', ' ', line).split(' ')[-3:], dtype=float) for line in tqdm(lines)]

spectrum = pd.DataFrame(np.array(extracted_data, dtype=float), columns=['e1KinE', 'e2KinE', 'P'])
spectrum['sum_KinE'] = spectrum['e1KinE'] + spectrum['e2KinE']

mask = (spectrum['sum_KinE'] * 1e3) < energy_threshold
spectrum = spectrum[mask]

spectrum['sum_KinE'] = np.round(spectrum['sum_KinE'] * 1e3)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3017195/3017195 [00:08<00:00, 356285.17it/s]


In [4]:
spectrum['sum_KinE'].unique()

array([ 3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.,
       16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
       29.])

In [5]:
file = './data/136Xe_hsd_cor.txt'
energy_threshold = 30

with open(file, 'r') as file:
    lines = file.readlines()
    extracted_data = [np.array(re.sub(' +', ' ', line).split(' ')[-2:], dtype=float) for line in tqdm(lines)]

angcor = pd.DataFrame(np.array(extracted_data, dtype=float), columns=['e1KinE', 'angle'])
angcor['e1KinE'] = angcor['e1KinE'] * 1e3
ang = interp1d(angcor['e1KinE'], angcor['angle'])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2457/2457 [00:00<00:00, 141986.84it/s]


In [6]:
def simulate(energy, size=int(1e4)):
    phase_space = deepcopy(spectrum[spectrum['sum_KinE'] == energy])
    phase_space['P'] = phase_space['P'] / phase_space['P'].sum()
    
    ind = np.random.choice(np.arange(len(phase_space)), p=phase_space['P'], size=size)
    sample = phase_space[['e1KinE', 'e2KinE']].iloc[ind]
    E1 = sample['e1KinE'].to_numpy() * 1e3
    E2 = sample['e2KinE'].to_numpy() * 1e3
    
    phi = 2 * np.pi * np.random.rand(size)
    a = ang(E1)
    cos = (-1 + np.sqrt(a**2 - 2 * a + 1 + 4 * a * np.random.rand(size))) / a
    sin = np.sqrt(1 - cos**2)
    D2x = sin * cos
    D2y = sin * sin
    D2z = cos
    
    output = []
    for i in range(size):
        output.append(f"e- {E1[i]} 1.0 0 0, e- {E2[i]} {D2x[i]} {D2y[i]} {D2z[i]}")
    with open(f'dbd/dbd_products_{int(energy)}.dat', 'w') as file:
        file.write('\n'.join(output))

In [8]:
for energy in range(3, 21):
    simulate(energy, size=int(1e4))